In [1]:
# !pip install beautifulsoup4
# !pip install requests
# !pip install pandas

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import json
import os
import datetime

In [3]:
def connection(page): # Функция соединения с сайтом
    url = "https://lenta.ru/parts/news/" + str(page)
    request = requests.get(url)
    return request

In [4]:
result_dict = {'date': [], 'href': [], 
               'title': [], 'about': []} # dict в который добавятся спарсенные значения 

print("Введите количество страниц: ")
limit = int(input()) # Количетсво страниц вебсайта, которые будут спарсены

print("Введите имя для нового файла: ")
file_name = input()+".csv"

page_num = 1 # Текущая страница
while page_num <= limit:

    soup = bs(connection(page_num).text, "html.parser") # Передача html текста 

    lenta_main_page = soup.find_all('li', class_='parts-page__item')
    lenta_main_title = soup.find_all('h3', class_='card-full-news__title')
    lenta_main_date = soup.find_all('time', class_='card-full-news__info-item card-full-news__date')



    for date in lenta_main_date: # Добавить дату в result_list словарь
        result_dict['date'].append(date.text)

    for link in lenta_main_page: # Добавить ссылки в result_list словарь
        links = link.a['href']
        if 'news/2022' in links:
            result_dict['href'].append('https://lenta.ru'+link.a['href'])
        elif '/parts/news' in links:
            page_num += 1
        else:
            result_dict['href'].append(link.a['href'])

    for head in lenta_main_title: # Добавить заголовки статей в result_list словарь
        result_dict['title'].append(head.text)


for href in result_dict['href']: # Добавить основной текст статьи в result_list словарь
    request = requests.get(href)
    txtsoup = bs(request.text, "html.parser")
    json_data = json.loads(txtsoup.find(type="application/ld+json").string)
    result_dict['about'].append(json_data['articleBody'])

    
df = pd.DataFrame(result_dict) # Сделать датафрейм из словаря result_dict


df.to_csv(file_name) # Сохранить файл формата .csv

if os.path.exists(file_name):
    print("Файл {0} успешно сохранён.".format(file_name))
    print("Размер:", os.path.getsize(file_name) // 1024, "Кб")
    print("Дата создания:",
          datetime.datetime.fromtimestamp(
              int(os.path.getctime(file_name))))
else:
    print("Объект не удалось сохранить.")

Введите количество страниц: 
5
Введите имя для нового файла: 
test
Файл test.csv успешно сохранён.
Размер: 223 Кб
Дата создания: 2022-09-16 11:50:19


In [5]:
data = pd.read_csv(file_name)
print("SHAPE = " + str(data.shape))
data.head()

SHAPE = (100, 5)


,Unnamed: 0,date,href,title,about
0,0,11:49,https://lenta.ru/news/2022/09/16/egypt/,Названы лучшие отели Египта для семейного отдыха,Туроператоры перечислили россиянам топ-5 гости...
1,1,11:45,https://lenta.ru/news/2022/09/16/nurmagomedov/,Нурмагомедов высказался о предстоящем бое Маха...,Бывший чемпион Абсолютного бойцовского чемпион...
2,2,11:45,https://lenta.ru/news/2022/09/16/himars_kherson/,Замглавы администрации Херсонской области прок...,Замглавы Херсонской области Кирилл Стремоухов ...
3,3,11:44,https://motor.ru/news/dacia-manifesto-16-09-20...,Dacia показала багги Manifesto со спальными ме...,Концепт Manifesto выступает своего рода «лабо...
4,4,11:44,https://motor.ru/news/chinese-cars-for-govt-16...,Российских чиновников начнут пересаживать на к...,Правительство Ленинградской области пересядет...
